In [1]:
import shutil
import os
import glob

In [2]:
test_id = [4, 6, 7, 13, 14, 19, 25, 29, 30, 31]

In [4]:
for i in range(1, 34):
    hazy_path = "/home/ubuntu/Competition/LowLevel/dense_haze/hazy"
    gt_path = "/home/ubuntu/Competition/LowLevel/dense_haze/GT"
    if i in test_id:
        shutil.copy(
            os.path.join(hazy_path, f"{i:02d}_hazy.png"),
            f"/home/ubuntu/Competition/LowLevel/dense_haze/test/input/{i:02d}.png",
        )
        shutil.copy(
            os.path.join(gt_path, f"{i:02d}_GT.png"),
            f"/home/ubuntu/Competition/LowLevel/dense_haze/test/gt/{i:02d}.png",
        )
    else:
        shutil.copy(
            os.path.join(hazy_path, f"{i:02d}_hazy.png"),
            f"/home/ubuntu/Competition/LowLevel/dense_haze/train/input/{i:02d}.png",
        )
        shutil.copy(
            os.path.join(gt_path, f"{i:02d}_GT.png"),
            f"/home/ubuntu/Competition/LowLevel/dense_haze/train/gt/{i:02d}.png",
        )

In [13]:
import os
import imageio.v3 as imageio
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from colour import (
    delta_E,
    Lab_to_XYZ,
    XYZ_to_Lab,
    RGB_to_XYZ,
    XYZ_to_RGB,
    RGB_COLOURSPACES,
)


def calculate_psnr(image_path1, image_path2):
    img1 = imageio.imread(image_path1)
    img2 = imageio.imread(image_path2)
    return psnr(img1, img2)


def calculate_ssim(image_path1, image_path2):
    img1 = imageio.imread(image_path1).astype(np.float32)
    img2 = imageio.imread(image_path2).astype(np.float32)
    return ssim(img1, img2, multichannel=True, channel_axis=2, data_range=255)


def calculate_ciede2000(image_path1, image_path2):
    img1 = imageio.imread(image_path1)
    img2 = imageio.imread(image_path2)

    # Convert images to Lab color space
    img1_lab = XYZ_to_Lab(RGB_to_XYZ(img1 / 255, RGB_COLOURSPACES["sRGB"]))
    img2_lab = XYZ_to_Lab(RGB_to_XYZ(img2 / 255, RGB_COLOURSPACES["sRGB"]))

    # Calculate CIEDE2000
    delta_E_value = delta_E(img1_lab, img2_lab, method="CIE 2000")
    return np.mean(delta_E_value)


def get_png_files(directory):
    return [f for f in os.listdir(directory) if f.endswith(".png")]


def calculate_metrics_for_directories(dir1, dir2):
    files1 = get_png_files(dir1)
    files2 = get_png_files(dir2)

    if set(files1) != set(files2):
        print("Warning: The files in the two directories do not match.")
        return

    psnr_values = []
    ssim_values = []
    ciede_values = []
    for file in files1:
        path1 = os.path.join(dir1, file)
        path2 = os.path.join(dir2, file)

        psnr_value = calculate_psnr(path1, path2)
        ssim_value = calculate_ssim(path1, path2)
        ciede_value = calculate_ciede2000(path1, path2)

        psnr_values.append(psnr_value)
        ssim_values.append(ssim_value)
        ciede_values.append(ciede_value)

        print(f"Metrics for {file}:")
        print(f"  PSNR: {psnr_value}")
        print(f"  SSIM: {ssim_value}")
        print(f"  CIEDE2000: {ciede_value}")

    average_psnr = np.mean(psnr_values)
    average_ssim = np.mean(ssim_values)
    average_ciede = np.mean(ciede_values)

    print(f"Average PSNR: {average_psnr}")
    print(f"Average SSIM: {average_ssim}")
    print(f"Average CIEDE2000: {average_ciede}")


# 设置路径
path1 = "/home/ubuntu/Competition/LowLevel/dense_haze/checkpoints/convnext_v2_huge_dehaze_dense/results"
path2 = "/home/ubuntu/Competition/LowLevel/dense_haze/val/gt"

# 计算 PSNR, SSIM 和 CIEDE2000
calculate_metrics_for_directories(path1, path2)

Metrics for 13.png:
  PSNR: 19.927303152913748
  SSIM: 0.5987331867218018
  CIEDE2000: 9.685345961706911
Metrics for 31.png:
  PSNR: 14.52653030242891
  SSIM: 0.5532832741737366
  CIEDE2000: 15.927072866718165
Metrics for 30.png:
  PSNR: 16.64928329115361
  SSIM: 0.5409196019172668
  CIEDE2000: 14.662910088061022
Metrics for 19.png:
  PSNR: 18.975258809485926
  SSIM: 0.4832790791988373
  CIEDE2000: 6.527299114709971
Metrics for 14.png:
  PSNR: 20.094547296536316
  SSIM: 0.5081163048744202
  CIEDE2000: 8.415752235870391
Metrics for 29.png:
  PSNR: 14.566273874923683
  SSIM: 0.47025883197784424
  CIEDE2000: 14.961785203509292
Metrics for 04.png:
  PSNR: 15.097994195434625
  SSIM: 0.4000838100910187
  CIEDE2000: 12.486304663568589
Metrics for 25.png:
  PSNR: 16.606658222349417
  SSIM: 0.4905698299407959
  CIEDE2000: 9.93306057313737
Metrics for 06.png:
  PSNR: 18.15632291501298
  SSIM: 0.5076149106025696
  CIEDE2000: 9.694495385147722
Metrics for 07.png:
  PSNR: 21.43754987818495
  SSIM: 

In [8]:
import colour